迭代器模式（iterator pattern）：惰性获取数据项，按需一次获取一个数据项。

yield 关键字：用于构建生成器（generator），其作用与迭代器一样。

所有生成器都是迭代器，因为生成器完全实现了迭代器的jiekou

内置的 range() 函数也返回一个类似生成器的对象。

在 Python 中，所有集合都可以迭代。在 Python 语言内部，迭代器用于支持：
- for 循环
- 构建和扩展集合类型
- 逐行遍历文本文件
- 列表推导、字典推导、集合推导
- 元组拆包
- 调用函数时，使用 * 拆包函数

In [1]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        # 返回字符串列表，非重叠匹配
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]

    def __len__(self):
        return len(self.words)

    def __repr__(self):
        # 生成大型数据结构的简略字符串表示形式，默认 30 个字符
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [2]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
s[0]

'The'

解释器需要迭代对象 x 时，会自动调用 iter(x) 。
- \_\_iter__
- \_\_getitem__
- TypeError

任何 Python 序列都可以迭代的原因是，它们都实现了 \_\_getitem__ 方法，标准的序列也都实现了 \_\_iter__ 方法。

鸭子类型（duck typing）的极端形式：只要实现特殊的 \_\_iter__ 方法，或者实现 \_\_getitem__ 方法且 \_\_getitem__ 方法的参数是从 0 开始的整数（int），就可以认为对象是可迭代的。


在白鹅类型（goose typing）理论中，如果是吸纳了 \_\_iter__ 方法，那么久认为对象是可迭代的。

检查对象 x 能否迭代，最准确的方法是：调用 iter(x) 函数，如果不可迭代，再处理 TypeError 异常。

Python 从可迭代的对象中获取迭代器

In [5]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [6]:
s = 'ABC'
it = iter(s)  # 构建迭代器
while True:
    try:
        print(next(it))  # 获取下一个元素
    except StopIteration:  # 元素耗尽，抛出异常
        del it  # 删除迭代器
        break

A
B
C


StopIteration 异常表明迭代器耗尽，Python 语言内部会处理 for 循环和其他迭代上下文（如列表推导、元素拆包，等等）中的 StopIteration 异常。

标准的迭代器接口有两个方法
- \_\_next__：返回下一个可用元素，如果没有元素了，抛出 StopIteration 异常
- \_\_iter__：返回 self ，以便在应该使用可迭代对象的地方使用迭代器

Iterator 抽象基类实现 \_\_iter__ 方法的方式是返回实例本身（return self）

检查对象 x 是否为迭代器最好的方式是调用 isinstance(x, abc,Iterator)

In [7]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)  # 获取迭代器
it

In [8]:
next(it)

'Pig'

In [9]:
next(it)

'and'

In [10]:
next(it)

'Pepper'

In [11]:
next(it)  # 元素耗尽，抛出异常

StopIteration: 

In [12]:
list(it)

[]

In [13]:
list(iter(s3))

['Pig', 'and', 'Pepper']

迭代器只需 \_\_next__ 和 \_\_iter__ 两个方法，所以除了调用 next() 方法，以及捕获 StopIteration 异常之外，没有办法检查是否还有遗留的元素。如果想再次迭代，那就要调用 iter(...) ，传入值钱构建迭代器的可迭代对象。

可迭代对象
- 使用 iter 内置函数可以获取迭代器的对象。如果对象实现了能返回迭代器的 \_\_iter__ 方法，那么对象就是可迭代的。序列都可以迭代；实现了 \_\_getitem__ 方法，而且其参数是从零开始的索引，这种对象也可以迭代。

迭代器
- 实现了无参数的 \_\_next__ 方法，返回序列中的下一个元素；如果没有元素，则抛出 StopIteration 异常。Python 中的迭代器还实现了 \_\_iter__ 方法，因此迭代器也可以迭代。

In [14]:
# 迭代器
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):  # 可迭代
        return SentenceIterator(self.words)  # 返回迭代器


class SentenceIterator:

    def __init__(self, words):
        self.words = words
        self.index = 0  # 索引

    def __next__(self):
        try:
            word = self.words[self.index]  # 索引上的元素
        except IndexError:
            raise StopIteration()  # 没有元素，抛出异常
        self.index += 1  # 下一个索引
        return word

    def __iter__(self):  # 返回 self
        return self

迭代器可以迭代，但是可迭代的对象不是迭代器。

为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个独立的迭代器，而且各个迭代器要能维护自身的内部状态。

可迭代的对象一定不能是自身的迭代器。即可迭代的对象必须实现 \_\_iter__ 方法，但不能实现 \_\_next__ 方法。

迭代器应该一直可以迭代，迭代器的 \_\_iter__ 方法应该返回自身。

In [15]:
# 生成器
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:  # 遍历
            yield word  # 产出
        # 不管有没有 return 语句，生成器函数都不会抛出 StopIteration 异常，而是在生成全部值之后直接退出
        return

只要 Python 函数的定义体中有 yield 关键字，该函数就是生成器函数。调用生成器函数时，会返回一个生成器对象。即生成器函数是生成器工厂。

In [16]:
# 生成器函数
def gen_123():
    yield 1 
    yield 2
    yield 3

# 函数对象
gen_123

<function __main__.gen_123()>

In [17]:
# 返回生成器对象
gen_123()

<generator object gen_123 at 0x00000217712EF580>

In [18]:
# 生成器是迭代器，会生成传给 yield 关键字的表达式的值
for i in gen_123():
    print(i)

1
2
3


In [19]:
# 生成器是迭代器
g = gen_123()
next(g)

1

In [20]:
next(g)

2

In [21]:
next(g)

3

In [22]:
next(g)

StopIteration: 

生层器函数会创建一个生成器对象，包装生成器函数的定义体。
- 把生成器传给 next(...) 函数时，生成器函数会向前，执行函数定义体中的下一个 yield 语句，返回产出的值，并在函数定义体的当前位置暂停。
- 最终，函数的定义体返回时，外层的生成器对象会抛出 StopIteration 异常
    - 这一点与迭代器协议一致
    
函数返回值；调用生成器函数返回生成器；生成器产出或生成值。生成器函数定义体中的 return 语句会触发生成器对象抛出 StopIteration 异常，只有把生成器函数当成协程使用时，这么做才有意义。

\_\_iter__ 方法是生成器函数，调用时会构建一个实现了迭代器接口的生成器对象。

In [23]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end.


惰性实现是指尽可能延后生成值，这样做能节省内存，而且或许还可以避免做无用的处理。

惰性求值（lazy evaluation）/及早求值（eager evaluation）

re.finditer 函数是 re.findall 函数的惰性版本，返回生成器，按需生成 re.MatchObject 实例。

In [24]:
# 惰性实现
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  # <1>

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):  # 迭代器
            yield match.group()  # 从 MatchObject 实例中提取匹配正则表达式的具体文本

生成器表达式可以理解为列表推导的惰性版本：不会迫切地构建列表，而是返回一个生成器，按需惰性生成元素。

In [25]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

# 列表推导
res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [26]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [27]:
# 生成器对象
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x00000217712BD430>

In [28]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


In [29]:
# 生成器表达式
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):  # 使用生成器表达式构建生成器，然后将其返回
        return (match.group() for match in RE_WORD.finditer(self.text))

生成器表达式是语法糖：完全可以替换成生成器函数，不过有时使用生成器表达式更便利。

生成器表达式是创建生成器的简洁语法。生成器函数灵活得多，可以使用多个语句实现复杂的逻辑，也可以作为协程使用。

如果生成器表达式要分成多行写，不如定义生成器函数，以便提高可读性。此外，生成器函数有名称，可以重用。

如果生成器表达式后面还有其他参数，那么必须使用括号围住，否则会抛出 SyntaxError 异常

```python
def __mul__(self, scalar):
    if isinstance(scalar, numbers.Real):
        return Vector(n * scalar for n in self)
    else:
        return NotImplemented
```

1. 生成器当作迭代器使用，从集合中获取元素
2. 用于生成不受数据源限制的值

内置的 range 函数用于生成有穷整数等差数列（Arithmetic Progression, AP），itertools.count 函数用于生成无穷等差数列。


In [30]:
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):  # <1>
        self.begin = begin
        self.step = step
        self.end = end  # None -> 无穷数列

    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)  # 强制类型转换  
        forever = self.end is None
        index = 0  # 降低处理浮点数时积累效应致错的风险
        while forever or result < self.end:  # 无穷
            yield result  # 生成值
            index += 1
            result = self.begin + self.step * index  # 计算可能存在的下一个结果

In [31]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [32]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [33]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [34]:
from fractions import Fraction

ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [35]:
from decimal import Decimal

ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [36]:
# 生成器函数
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

In [37]:
ap = aritprog_gen(0, 1, 3)
list(ap)

[0, 1, 2]

itertools.count 函数从不停止；itertools.takewhile 函数会生成一个使用另一个生成器的生成器，在指定条件计算结果为 False 时停止。因此可以结合使用。

In [38]:
import itertools

gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [39]:
# 生成器工厂函数，返回生成器
# 不是生成器函数，没有 yield 关键字
def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

通用的函数：参数为任意的可迭代对象，返回值是生成器，用于生成选中的、计算出的和重新排列的元素。

1. 用于过滤的生成器函数

    - 从输入的可迭代对象中产出元素的子集，而且不修改元素本身。
    - 大多数都接受一个断言参数（predicate），布尔函数，有一个参数，会应用到输入中的每个元素上，用于判断元素是否包含在输出中

In [40]:
# 过滤
def vowel(c):
    return c.lower() in 'aeiou'

In [41]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [42]:
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [43]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [44]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [45]:
list(itertools.compress('Aardvark', (1,0,1,1,0,1)))

['A', 'r', 'd', 'a']

In [46]:
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [47]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [48]:
list(itertools.islice('Aardvark', 1, 7 ,2))

['a', 'd', 'a']

2. 用于映射的生成器函数

    - 在输入的单个可迭代对象（map 和 starmap 函数处理多个可迭代的对象）中的各个元素上做计算，然后返回结果
    

In [49]:
# 映射
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

In [50]:
list(itertools.accumulate(sample))  # 总和

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [51]:
list(itertools.accumulate(sample, min))  # 最小值

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [52]:
list(itertools.accumulate(sample, max))  # 最大值

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [53]:
import operator
list(itertools.accumulate(sample, operator.mul))  # 乘积

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [54]:
list(itertools.accumulate(range(1, 11), operator.mul))  # 阶乘

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [55]:
list(enumerate('albatroz', 1))  # 编号

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [56]:
list(map(operator.mul, range(11), range(11)))  # 平方

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [57]:
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [58]:
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [59]:
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [60]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.starmap(lambda a, b: b/a,
                      enumerate(itertools.accumulate(sample), 1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

3. 用于合并的生成器函数

    - 从输入的多个可迭代对象中产出元素

In [61]:
# 合并
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [62]:
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [63]:
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [64]:
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [65]:
list(zip('ABC', range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [66]:
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [67]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [68]:
# itertools.product 生成器是计算笛卡尔积的惰性方式
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [69]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [70]:
# 传入可迭代对象，产出一系列只有一个元素的元组
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [71]:
# 重复 2 次处理输入的各个可迭代对象
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [72]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [73]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows:
    print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


4. 有些生成器函数会从一个元素中产出多个值，扩展输入的可迭代对象

In [74]:
# 创建生成器
ct = itertools.count()
next(ct)

0

In [75]:
next(ct), next(ct), next(ct)

(1, 2, 3)

In [76]:
# 增加限制才能构造列表
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [77]:
# 创建生成器
cy = itertools.cycle('ABC')
next(cy)

'A'

In [78]:
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [79]:
# 创建生成器
rp = itertools.repeat(7)
next(rp), next(rp)

(7, 7)

In [80]:
# 第二个 times 参数 可以限制生成器生成的元素数量
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [81]:
# repeat 函数常见用途：为 map 函数提供固定参数
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

combinations、combinations_with_replacement、permutations 生成器函数，连同 product 函数，成为组合学生成器（combinatoric generator）。

In [82]:
list(itertools.combinations('ABC', 2))  # 组合

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [83]:
list(itertools.combinations_with_replacement('ABC', 2))  # 组合

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [84]:
list(itertools.permutations('ABC', 2))  # 排列

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [85]:
list(itertools.product('ABC', repeat=2))  # 笛卡尔积

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

5. 有些生成器函数用于产出输入的可迭代对象中的全部元素，不过会以某种方式重新排列

内置的 reversed 函数是其中唯一不接受可迭代的对象，只接收序列为参数的函数。因为 reversed 函数从后向前产出元素，只有序列的长度已知时才能工作。

itertools.groupby 假定输入的可迭代对象要使用分组标准排序；即使不排序，至少也要使用指定的标准分组各个元素。处理 groupby 函数返回的生成器要嵌套迭代，为了使用 groupby 函数，要排序输入。

itertools.tee 从输入的一个可迭代对象中产出多个生成器，每个生成器都可以产出输入的各个元素。

In [86]:
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x217712c8e20>),
 ('A', <itertools._grouper at 0x217712c8c40>),
 ('G', <itertools._grouper at 0x217712c8970>)]

In [87]:
for char, group in itertools.groupby('LLLLAAGGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A']
G -> ['G', 'G', 'G']


In [88]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear',
          'bat', 'dolphin', 'shark', 'lion']
animals.sort(key=len)
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [89]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [90]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [91]:
list(itertools.tee('ABC'))

[<itertools._tee at 0x21771306440>, <itertools._tee at 0x217713065c0>]

In [92]:
g1, g2 = itertools.tee('ABC')
next(g1)

'A'

In [93]:
next(g2)

'A'

In [94]:
next(g2)

'B'

In [95]:
list(g1)

['B', 'C']

In [96]:
list(g2)

['C']

In [97]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

除了代替循环之外，yield from 还会创建通道，把内层生成器直接与外层生成器的客户端联系起来。

In [98]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [99]:
def chain(*iterables):
    for i in iterables:
        yield from i
    
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

归约/合拢/累加函数：接受一个可迭代的对象，然后返回单个结果。

对 all 和 any 函数来说，有一项重要的优化措施是 reduce 函数做不到的：这两个函数会短路（一旦确定了结果就立即停止使用迭代器）。

sotred 会构建并返回真正的列表，可以处理任意的可迭代对象；reversed 是生成器函数。

sorted 和归约函数只能处理最终会停止的可迭代对象。

In [100]:
all([1, 2, 3])

True

In [101]:
all([1, 0, 3])

False

In [102]:
all([])

True

In [103]:
any([1, 2, 3])

True

In [104]:
any([1, 0, 3])

True

In [105]:
any([0, 0.0])

False

In [106]:
any([])

False

In [107]:
g = (n for n in [0, 0.0, 7, 8])
any(g)

True

In [108]:
next(g)

8

iter 函数有一个鲜为人知的用法：传入两个参数，使用常规的函数或任何可调用的对象创建迭代器。
- 第一个参数必须是可调用的对象，用于不断调用（没有参数）
- 第二个参数是哨符，这是个标记值。当可调用的对象返回这个值时，触发迭代器抛出 StopIteration 异常，而不产出哨符

In [109]:
def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1)
d6_iter

In [110]:
# d6_iter 一旦耗尽就没用了
for roll in d6_iter:
    print(roll)

NameError: name 'randint' is not defined

In [111]:
with open('Romeo and Juliet.txt') as fp:
    for line in iter(fp.readline, '\n'):
        print(line)

Romeo and Juliet | Act 5, Scene 3



读写过程必须采用渐进方式，因为完整的数据集比主内存大得多。解决方法很简单：主 for 循环每次迭代时从文件中读取一个记录，转换后将其写入文件。

利用生成器解耦读逻辑和写逻辑，使用生成器可以交叉读写。

使用生成器处理数据库时，我们把记录看成数据流，这样消耗的内存量最低，而且吧㕆数据有多大都能处理。

生成器对象的 .\_\_send__ 方法
- 与 .\_\_next__() 方法一样，.send() 方法使生成器前进到下一个 yield 语句
- .send() 方法允许在客户端代码和生成器之间双向交换数据
    - .\_\_next__() 方法 只允许客户从生成器中获取数据

设计模式在各种编程语言中使用的方式并不相同，Python 从语义上集成迭代器模式是个很好的例证。

以 iter(o) 的形式调用时返回的是迭代器，以 iter(func, sentinel) 的形式调用时，能使用任何函数构造迭代器。

In [112]:
# 生成器
def f():
    x = 0
    while True:
        x += 1
        yield x

yield 关键字只能把最近的外层函数变成生成器函数。虽然生成器函数看起来像函数，但是我我们不能通过简单的函数调用把职责委托给另一个生成器函数。

In [113]:
# 错误的抽象
def f():
    def do_yield(n):
        yield n
    x = 0
    while True:
        x += 1
        do_yield(x)

yield form 句法允许生成器或协程把工作委托给第三方完成，这样就无需嵌套 for 循环作为变通了。

In [114]:
# 正确的抽象
def f():
    def do_yield(n):
        yield n
    x = 0
    while True:
        x += 1
        yield from do_yield(x)

生成器与迭代器的语义对比

1. 接口

    - Python 的迭代器定义了两个方法：\_\_next__ 和 \_\_iter__
    - 生成器对象实现了这两个方法，因此从这方面来看，所有生成器都是迭代器

In [115]:
from collections import abc

e = enumerate('ABC')
isinstance(e, abc.Iterator)

True

2. 实现方式

    - 生成器这种 Python 语言结构可以使用两种方式编写：含有 yield 关键字的函数、生成器表达式
    - 调用生成器函数或者执行生成器表达式得到的生成器对象属于语言内部的 GeneratorType 类型
        - GeneratorType 类型的实例实现了迭代器接口，因此从这方面来看，所有生成器都是迭代器
    - 我们可以编写不是生成器的迭代器，方法是实现经典的迭代器模式，或使用 C 语言编写扩展
        - 从这方面来看，enumerate 对象不是生成器
        
types.GeneratorType 类型：生成器-迭代器对象的类型，调用生成器函数时生成

In [116]:
import types
e = enumerate('ABC')
isinstance(e, types.GeneratorType)

False

3. 概念

    - 迭代器用于遍历集合，从中产出元素
    - 调用 next(it) 时，迭代器不能修改从数据源中读取的值，只能原封不动地产出值
    - 生成器可能无需遍历集合就能生成值
    - 即使依附了集合，生成器不仅能产出集合中的元素，还可能会产出派生自元素的其他值
    
    实现方式无关紧要，不使用 Python 生成器对象也能编写生成器

In [117]:
# 愚蠢的示例
class Fibonacci:

    def __iter__(self):
        return FibonacciGenerator()


class FibonacciGenerator:

    def __init__(self):
        self.a = 0
        self.b = 1

    def __next__(self):
        result = self.a
        self.a, self.b = self.b, self.a + self.b
        return result

    def __iter__(self):
        return self

In [118]:
# 符合 Python 风格的生成器
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

在生成器的定义中，迭代器和生成器是同义词，生成器指代生成器函数，以及生成器函数构建的生成器对象
- [术语对照表](https://docs.python.org/zh-cn/3/glossary.html)